In [9]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms ,models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import numpy as np
from spectral import envi, principal_components
from tqdm import tqdm
import random
import albumentations as A
import shutil



In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder('aug_processed_data', transform=transform)

# Train/val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
num_classes = 2

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # 64x64
        x = self.pool(F.relu(self.conv2(x)))   # 32x32
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [6]:
## Approch 2


class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(p=0.5)  # Dropout with 50% probability

        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))   # 128->64
        x = self.pool(F.relu(self.bn2(self.conv2(x))))   # 64->32

        x = x.view(-1, 32 * 32 * 32)
        x = self.dropout(F.relu(self.fc1(x)))            # Dropout before fc2
        x = self.fc2(x)
        return x

In [7]:
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # weight_decay added

In [8]:
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy


def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            print(f"Batch loss: {loss.item()}")
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = total_loss / len(train_loader)
        train_acc = 100 * correct / total

        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

# Assuming you already have a val_loader for your validation dataset

train_model(epochs=100)

Batch loss: 0.7209358215332031
Batch loss: 3.0623459815979004
Batch loss: 5.982723236083984
Batch loss: 5.07076358795166
Batch loss: 5.961067199707031
Epoch 1 | Train Loss: 4.1596 | Train Acc: 56.88% | Val Loss: 0.6107 | Val Acc: 67.50%
Batch loss: 4.462386608123779
Batch loss: 2.5849502086639404
Batch loss: 3.0387144088745117
Batch loss: 1.0572116374969482
Batch loss: 1.0128306150436401
Epoch 2 | Train Loss: 2.4312 | Train Acc: 73.12% | Val Loss: 0.9145 | Val Acc: 77.50%
Batch loss: 1.0918093919754028
Batch loss: 1.1405127048492432
Batch loss: 0.6669681072235107
Batch loss: 0.6395519971847534
Batch loss: 1.911686897277832
Epoch 3 | Train Loss: 1.0901 | Train Acc: 80.62% | Val Loss: 0.4056 | Val Acc: 87.50%
Batch loss: 0.2729165554046631
Batch loss: 0.13588377833366394
Batch loss: 0.2925509512424469
Batch loss: 0.33915093541145325
Batch loss: 0.24996091425418854
Epoch 4 | Train Loss: 0.2581 | Train Acc: 91.25% | Val Loss: 0.6128 | Val Acc: 77.50%
Batch loss: 0.058132294565439224
Batch 

In [10]:
# === PARAMETERS ===
root_folder = 'aug_processed_data'  # Folder with 'healthy' and 'unhealthy' subfolders
base_dir = 'split_dat'                   # New folder where train/val split will go
train_ratio = 0.8                         # 80% train, 20% val split

# === 1. Create train/val folders with same subfolder structure ===
def create_train_val_split(root_folder, base_dir, train_ratio=0.8):
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)  # Clean previous split if exists
    os.makedirs(base_dir)

    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')

    os.makedirs(train_dir)
    os.makedirs(val_dir)

    classes = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls))
        os.makedirs(os.path.join(val_dir, cls))

        images = os.listdir(os.path.join(root_folder, cls))
        random.shuffle(images)

        train_count = int(len(images) * train_ratio)
        train_imgs = images[:train_count]
        val_imgs = images[train_count:]

        for img_name in train_imgs:
            src = os.path.join(root_folder, cls, img_name)
            dst = os.path.join(train_dir, cls, img_name)
            shutil.copy(src, dst)

        for img_name in val_imgs:
            src = os.path.join(root_folder, cls, img_name)
            dst = os.path.join(val_dir, cls, img_name)
            shutil.copy(src, dst)

    print(f"✅ Dataset split done! Train and val folders created at '{base_dir}'")

# === Run the split function ===
create_train_val_split(root_folder, base_dir, train_ratio)

# === 2. Setup Transfer Learning with the split folders ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze backbone

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.classes))  # Number of classes

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# === Training loop ===
def train_model(epochs=10):
    best_val_acc = 0.0
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_loss = total_loss / len(train_loader)

        model.eval()
        val_correct, val_total, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc

        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    torch.save(model.state_dict(), "plant_model.pth")
    print(f"\n🏆 Best Val Accuracy: {best_val_acc:.2f}%")

# === Run training ===
train_model(epochs=10)


✅ Dataset split done! Train and val folders created at 'split_dat'
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/codespace/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 125MB/s] 


Epoch 1 | Train Loss: 0.8098 | Train Acc: 49.38% | Val Loss: 0.6427 | Val Acc: 55.00%
Epoch 2 | Train Loss: 0.6459 | Train Acc: 66.88% | Val Loss: 0.5559 | Val Acc: 62.50%
Epoch 3 | Train Loss: 0.5841 | Train Acc: 72.50% | Val Loss: 0.5622 | Val Acc: 75.00%
Epoch 4 | Train Loss: 0.4728 | Train Acc: 78.12% | Val Loss: 0.4818 | Val Acc: 85.00%
Epoch 5 | Train Loss: 0.4298 | Train Acc: 84.38% | Val Loss: 0.4492 | Val Acc: 85.00%
Epoch 6 | Train Loss: 0.3756 | Train Acc: 86.88% | Val Loss: 0.4484 | Val Acc: 87.50%
Epoch 7 | Train Loss: 0.3404 | Train Acc: 88.12% | Val Loss: 0.4034 | Val Acc: 85.00%
Epoch 8 | Train Loss: 0.3033 | Train Acc: 93.75% | Val Loss: 0.4014 | Val Acc: 87.50%
Epoch 9 | Train Loss: 0.2892 | Train Acc: 93.12% | Val Loss: 0.4137 | Val Acc: 87.50%
Epoch 10 | Train Loss: 0.2579 | Train Acc: 95.00% | Val Loss: 0.3797 | Val Acc: 90.00%

🏆 Best Val Accuracy: 90.00%
